### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [3]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
def combine_data(df_o, df_inv, df_his):
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    for date_current in df_o.index:
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

In [6]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [7]:

'''
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 이쓴
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml = '../data/data_for_ml/'
pkl_directory = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle(pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    globals()['df_{}_{}'.format(val[1], 'combined')] = combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )
                            
'''

"\n# investor.pkl, historical.pkl. 읽기\n\n# ****************************************************************************************************\n## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.\n##   C:\\jupyter연습\x0cactors_affecting_stock_price' 디렉토리에 이쓴\n#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)\n#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)\n# ****************************************************************************************************\ndirectory_for_ml = '../data/data_for_ml/'\npkl_directory = '../data/company_pkl/'\nfor key, val in code.items():\n    pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])\n    globals()['df_{}'.format(val[1])] = pd.read_pickle(pkl_name)\n    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp\n    globals()['df_{}'.format(val[1])]['date']

In [13]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 이쓴
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml = '../data/data_for_ml/'
pkl_directory = '../data/company_pkl/'

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday', 'value' ]

for key, val in code.items():
    pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
    df_class_temp = pd.read_pickle(pkl_name)
    df_class_temp['date'] = pd.to_datetime(df_class_temp['date']) #change to timestamp
    df_class_temp['date'] = df_class_temp['date'].dt.date  # change to datetime
    
    pkl_name= '{}_historical.pkl'.format(val[1])
    df_historical_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_historical_temp = df_historical_temp[hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    df_historical_temp['date'] = df_historical_temp['date'].dt.date # change to datetime
    
    pkl_name= '{}_investors.pkl'.format(val[1])
    df_investors_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_investors_temp['date'] = df_investors_temp['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    df_combinde_temp = combine_data(df_class_temp.set_index('date'), df_investors_temp.set_index('date'),
                                    df_historical_temp.set_index('date'))
    
    globals()['df_{}_{}'.format(val[1], 'combined')] = df_combinde_temp.copy()
    globals()['df_{}_{}'.format(val[1], 'sel')] = df_combinde_temp[column_selected]

In [10]:
df_ssbio_combined.head()

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc,open,high,low,close,vol,weekday,value
2022-06-20,-0.009162,-0.633554,17.785377,-6.154967,-3.010363,59.274510,-11.126582,inf,-0.765013,-1.000000,-0.365994,-1.153571,-0.012422,0.022113,-0.007491,0.030979,3.355708,4,0
2022-06-21,-0.582959,-1.065342,-0.226993,1.034687,-0.753436,-0.996747,-0.243750,-1.000000,-5.038889,NaN,-0.114773,4.953488,0.037736,-0.007212,0.015094,-0.012019,-0.788559,0,0
2022-06-22,-0.260879,-9.685811,-0.861459,-0.323273,-5.320557,-389.900000,-1.183471,inf,-1.022696,-inf,-1.138639,-1.136719,0.006061,0.004843,0.009913,-0.001217,-0.273738,1,0
2022-06-23,1.028595,1.192143,0.676436,-0.830192,0.284677,-1.537927,-0.900901,-1.019724,-0.939394,-1.620690,-0.092593,1.085714,-0.014458,-0.006024,-0.006135,-0.004872,0.74693,2,0
2022-06-24,1.151050,-1.848829,14.423776,11.329670,-2.192718,3.236616,61.000000,60.300000,39.000000,-5.166667,-18.265306,-1.219178,-0.003667,0.004848,0.002469,0.003672,0.462932,3,0


In [8]:
'''
# column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
#               'open', 'high', 'low', 'close',  'vol', 'weekday', 'value']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday', 'value' ]
'''

## 주요 지표 읽기

### Read and Combine data 

In [10]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [14]:
# 매일 개장일자에 따른 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [15]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [16]:
pkl_common_directory = '../data/common_pkl/'
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# for index_name in major_index:
for index_name in common_pkl:
    key_name = index_name[:-4]
#     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
#     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
                            how='left', left_index=True, right_index=True)

In [17]:
df_base.tail()

,dji_cr,dji_f_cr,dxy_cr,ixic_f_cr,bond_kor_10_cr,bond_kor_2_cr,kosdaq_cr,kospi_cr,krw_cr,ixic_cr,spx_f_cr,sox_cr,spx_cr,bond_usa_10_cr,bond_usa_2_cr,bond_usa_3m_cr,vix_cr,wti_cr
date,,,,,,,,,,,,,,,,,,
2023-02-25,NaN,NaN,NaN,NaN,-0.40%,-0.79%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-26,NaN,+0.05%,NaN,+0.17%,0.00%,0.00%,NaN,NaN,+0.03%,NaN,+0.09%,NaN,NaN,-0.30%,-0.13%,+0.05%,NaN,+0.43%
2023-02-27,+0.22%,+0.21%,-0.53%,+0.55%,+2.50%,+3.62%,+0.18%,-0.87%,+0.30%,+0.63%,+0.21%,+0.55%,+0.38%,-0.34%,-0.43%,+0.72%,-3.32%,-1.19%
2023-02-28,NaN,+0.14%,0.13%,+0.28%,+2.16%,+1.75%,NaN,+0.87%,-0.18%,NaN,+0.18%,NaN,NaN,+0.10%,+0.13%,-0.21%,0.00%,-0.20%
2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### combine with 'df_xxx_combined'

In [15]:
'''
df_sec_sel = df_sec_combined[column_selected]
df_hyunmotor_sel = df_hyunmotor_combined[column_selected]
df_naver_sel = df_naver_combined[column_selected]
df_ktng_sel = df_ktng_combined[column_selected]
'''

In [18]:
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df_base, how='left', left_index=True, right_index=True)

In [16]:
'''
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)
'''

In [19]:
# 'close_cr' column 추가
pkl_d = '../data/company_pkl/'
for key, val in code.items():
    pkl_name= '{}_historical.pkl'.format(val[1])
#     df = pd.read_pickle(pkl_d + pkl_name).set_index('date')[['close_cr']]
    df = pd.read_pickle(pkl_d + pkl_name).set_index('date')
    df = df.apply(pd.to_numeric) 
    min_rate = 0.005 # 수수료등 비용 0.2672%
    df['close_cr_5'] = df['close_cr'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    # 전일대비 +min_rate 이상이면 1, 이하이면  0으로 변환  
    df['r_open_high'] = (df['high'] - df['open'])/df['open'] # 시작가격 이후에 최고가가 min_rate 이상이면 1, 아니면 0
    df['r_open_high_5'] = df['r_open_high'].map(lambda x : 0 if x < min_rate else ( 1 if x >= min_rate  else  "" ))
    
    min_rate = 0.01 # 수수료등 비용 0.2672%
    df['close_cr_1'] = df['close_cr'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    # 전일대비 +min_rate 이상이면 1, 이하이면  0으로 변환  
    df['r_open_high_1'] = df['r_open_high'].map(lambda x : 0 if x < min_rate else ( 1 if x >= min_rate  else  "" ))
    df =  df[['close_cr_5', 'r_open_high_5', 'close_cr_1', 'r_open_high_1']]
#     # # 시작가이후에 최고가가 나오는지 high가 양수면 1, 음수면 0
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df, how='left', left_index=True, right_index=True)

In [20]:
pd.read_pickle(pkl_d + pkl_name).head()

,date,open,high,low,close,close_cr,vol
0,2020-01-01,,,,,,
1,2020-01-02,93800.0,93900.0,92400.0,92500.0,-1.39,41620.0
2,2020-01-03,92500.0,93200.0,92500.0,92500.0,0.0,31176.0
3,2020-01-04,,,,,,
4,2020-01-05,,,,,,


In [21]:
# new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity', 'open',
#   'high', 'low', 'close', 'vol', 'dji_cr', 'ixic_cr', 'spx_cr', 'kosdaq_cr', 'kospi_cr', 'weekday', 'value', 'close_cr']

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust',
                'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                'corporateetc', 'foreigneretc', 'open', 'high', 'low', 'close', 'vol',
                'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr',
                'bond_kor_10_cr', 'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
                'kosdaq_cr', 'kospi_cr', 'krw_cr',
                'sox_cr', 'vix_cr', 'wti_cr', 
                'weekday', 'value', 'close_cr_5', 'r_open_high_5', 'close_cr_1', 'r_open_high_1']

In [ ]:
'''
# column 순서 변경 : weekday, value column을 마지막으로 이동 
df_sec_sel =  df_sec_sel[new_columns]
df_hyunmotor_sel =  df_hyunmotor_sel[new_columns]
df_naver_sel =  df_naver_sel[new_columns]
df_ktng_sel =  df_ktng_sel[new_columns]
'''

In [22]:
# column 순서 변경 : weekday, value, 'close_cr_5', 'r_open_high_5', 'close_cr_1', 'r_open_high_1' column을 마지막으로 이동 
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = globals()['df_{}_sel'.format(val[1])][new_columns]

In [ ]:
'''
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_hyunmotor_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_naver_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
df_ktng_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)
'''

In [23]:
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].replace([np.inf, -np.inf], [1, -1], inplace=True)

In [ ]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
df_sec_sel.dropna(inplace=True)
df_hyunmotor_sel.dropna(inplace=True)
df_naver_sel.dropna(inplace=True)
df_ktng_sel.dropna(inplace=True)

In [24]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].dropna(inplace=True)

In [25]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)
'''
df_sec_sel = string_to_num(df_sec_sel)
df_hyunmotor_sel = string_to_num(df_hyunmotor_sel)
df_naver_sel = string_to_num(df_naver_sel)
df_ktng_sel = string_to_num(df_ktng_sel)
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = string_to_num(globals()['df_{}_sel'.format(val[1])])

In [27]:
# pickle로 데이터 저장
'''
directory_for_ml = '../data/data_for_ml/'
df_sec_sel.to_pickle(directory_for_ml + 'df_sec_sel.pkl')
df_hyunmotor_sel.to_pickle(directory_for_ml + 'df_hyunmotor_sel.pkl')
df_naver_sel.to_pickle(directory_for_ml + 'df_naver_sel.pkl')
df_ktng_sel.to_pickle(directory_for_ml + 'df_ktng_sel.pkl')

df_sec_sel.to_csv(directory_for_ml + 'df_sec_sel.csv')
df_hyunmotor_sel.to_csv(directory_for_ml + 'df_hyunmotor_sel.csv')
df_naver_sel.to_csv(directory_for_ml + 'df_naver_sel.csv')
df_ktng_sel.to_csv(directory_for_ml + 'df_ktng_sel.csv')
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].to_pickle(directory_for_ml + 'df_{}_sel.pkl'.format(val[1]))
    globals()['df_{}_sel'.format(val[1])].to_csv(directory_for_ml + 'df_{}_sel.csv'.format(val[1]))

### feature 이름 설명
#### feature에 사용된 column name 설명

In [ ]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [ ]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [ ]:
# financial이 무엇인지 알기 위한 명령
c_name = 'dxy_cr'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'cpi'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'sox'
standforK(c_name), standforE(c_name), standforexp(c_name)

### 설명에서 feature name 찾기

In [ ]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [ ]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

In [ ]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

#### pd.merge 형식 참조
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)

In [ ]:
bokd = '../data/common_pkl/bok_rate.pkl'
cpid = '../data/common_pkl/cpi.pkl'
djid = '../data/common_pkl/dji.pkl'
bok = pd.read_pickle(bokd)
cpi = pd.read_pickle(cpid)
dji = pd.read_pickle(djid)
boks = pd.read_pickle(bokd).set_index('date')
cpis = pd.read_pickle(cpid).set_index('date')
djis = pd.read_pickle(djid).set_index('date')

In [ ]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [ ]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [ ]:
df_base = df_base.merge(cpis, how='left', left_index=True, right_index=True)

In [ ]:
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)